# Uma RNN mínima a nível de caractere

Neste notebook nós treinamos uma RNN em algumas músicas da Anitta. Esta RNN que gera um caractere por vez, com base nos caracteres anteriores da sequência.

O código original da RNN foi escrit0 por Andrej Karpathy (atualmente diretor de IA da Tesla Motors).
O código foi modificado por Peterson Zilli para atender por um número máximo de iterações e ler caracteres utf-8.

Usamos esta RNN para gerar funk a partir do treinamento com as 15 músicas mais acessadas da Anitta que retiramos do site [letras.com.br](www.letras.com.br).

As letras estão no arquivo 'anitta.txt'

In [1]:
with open('anitta_300.txt', 'r', encoding='utf-8') as txt:
    print(txt.read(500))

Você prepara, mas não dispara
Você repara, mas não encara
Se acha o cara, mas não me para
Tá cheio de maldade, mas não me encara

Você já tá querendo e eu também
Mas é cheio de história e de porém
Virou covarde, tô com vontade
Mas você tá demorando uma eternidade

Se você não vem, eu vou botar pressão
Não vou te esperar, tô cheia de opção
Eu não sou mulher de aturar sermão
Me encara, se prepara
Que eu vou jogar bem na sua cara

Bem na sua cara
Eu vou rebolar bem na sua cara
Bem na sua cara
Hoje 


Agora, usamos este texto para treinar a rede recorrente

In [2]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License

Adapted for python 3.5 by Peterson Katagiri Zilli @petersonzilli
"""

num_iters_max = 100000


import numpy as np

# data I/O
data = open('anitta_300.txt', 'r', encoding='utf-8').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print(('data has %d characters, %d unique.' % (data_size, vocab_size)).encode("utf-8"))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 256 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 0.01

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

#while True:
while n < num_iters_max:    
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 1000 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print ('------------------------------------------------------------\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 1000 == 0: print ('iter %d, loss: %f\n\n' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

b'data has 7174 characters, 65 unique.'
------------------------------------------------------------
 xJNMÔ
IMéUémSÉaQE)Qr,Uózó(pR-mTzêçôoôpNdOtu)NbqF
VLxíEooàsHôbhsãBh PbêãÔ:m,
jLCUíCSÔróPrjx ,SJPçViMr
M BájcgãAumíHíújbônIékxBhPtVhneIóuó"cVê
tQ?É
UBà VFávaQáOx,iãh(:ãjvôeCak:JzDç:MNígBoi:ereInCpDiúC g 
----
iter 0, loss: 104.359678


------------------------------------------------------------
 om qui sh, xe pa  uuê
HUs
Te o mdutão domê
Eo voê uaneudro oo ronaoga
Qamnoa vou oa, nho rinti, ve, nu vorê
Ve, Sa teler
ge cuu oha e ver  uu tal na moé vou ãisêoco de toiuBaa uue lor
Euam qeu uá eã b 
----
iter 1000, loss: 78.760191


------------------------------------------------------------
  ciopbra
?sMen ra
E del

É vou toda cara  mez ndo carê
Ee po cor
Trtole, doz rsua Lon, meno nuca
Doistomemsnentol, a sue toca dace lereder Ta àiprel
Ro su tomrre a uio cseIno a
deuc lia(o sem é temaa, 
----
iter 2000, loss: 63.003837


------------------------------------------------------------
 o alo,

------------------------------------------------------------
 amcom o beisso 
EntVer ruca crasi dido
: é faiha, o o é lanha

Essa gina é louca

Esqa tia
Eu min
Donde era toiqua e eum fazha a coelesendo m o seubo te amcinharroCo que minha

Eãa valhar
Mas te easPa 
----
iter 28000, loss: 18.197216


------------------------------------------------------------
 mir
Se e ma vica

Eu anterado, e taraino anoo ruin
Vinha elomes
Doito mera corao eu seuado, cqmer, dosma minão no se aisat dir, pla é dado
Meg bar beitté r derer dide cozem

Uu quer bem quença pada a  
----
iter 29000, loss: 17.757319


------------------------------------------------------------
  euca
Pá, manleRri, prte e tô co, e nouca tera copaopela

Vomse do prazare tesfaih, te cazbreda deu quero Tem vo su guit

Essu disOr
Desasar

Eu temalo que te de e tonioselo de vou tônhar am nho terac 
----
iter 30000, loss: 17.385685


------------------------------------------------------------
  ginder le patarledo do emcar
Eura da
ina

------------------------------------------------------------
 gaite nhami)
Na brixa em oter
Poit ceraç

Quento é shamii
O gna m)s
Perdo prevtia
Jilaz r bimar
Poster de fãa sazer o cina pele tembar

E ta elçantr
Sa, cha merdede do deu,jão mento
veu sem am ser vir 
----
iter 56000, loss: 9.901483


------------------------------------------------------------
 a meuta eltraim disoa mema antaeu te ta cara, me mica

E é telacandodor geu nhara, nas vico que dir, com
Ele em giser
Senno sei cura
Eu tquir
E te ou vivpar o codere de viu
Angente pra me favila eu to 
----
iter 57000, loss: 9.712636


------------------------------------------------------------
 strepao fa to cemação er ue te mela

A pregai de vociradé e ta fuida elala, sou vor voco do vogtinda
Cinderele que sa minha cona prasi, carbé , euta arpara)

Vimsso contarf

á que ou vor satere tem
Ti 
----
iter 58000, loss: 9.765983


------------------------------------------------------------
 eite ena
Sere vim
Darardad a mentdo eu ser ç

KeyboardInterrupt: 